In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
r = requests.get("http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c = r.content
soup = BeautifulSoup(c, 'html.parser')

# Understanding the webpage through inspect 
# Use inspect to determine the division and header tags used to contain the information you want to scrape 
# discovered that h4 class propPrice contain the price of the house
# discovered that div class CardDetails contains the rest of the information we want 

all = soup.find_all("div", {"class":"propertyRow"})
#print(len(all)) #because this is a list we can apply manipulation  
# taking a look at the first entry 
print(all[0].find("h4", {"class":"propPrice"}).text.replace("\n","").replace(" ", "")) # because ofhow the price is displayed, we replace all the weird extra stuff with empty space 

page_nr = soup.find_all('a', {'class':'Page'})[-1].text
print(page_nr)

$725,000
3


In [3]:
list = [] 
base_url = "http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
for page in range(0,int(page_nr)*10,10): 
    print(base_url+str(page)+'.html') 
    r = requests.get(base_url+str(page)+'.html',headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
    c = r.content 
    soup = BeautifulSoup(c, "html.parser")
    all = soup.find_all('div', {'class':'propertyRow'})
    for item in all: 
        d = {} # Storing info into dictionary to convert to dataframe 
        d['Price'] = item.find("h4", {"class":'propPrice'}).text.replace('\n','').replace(' ','')
        d['Address'] = item.find_all("span", {"class" : 'propAddressCollapse'})[0].text
        try: 
            d['Locality'] = item.find_all("span", {"class" : 'propAddressCollapse'})[1].text
        except: 
            d['Locality'] = None
        # Try and except is used to return 0 for entries that do not have that specific info
        try:
            d['Beds'] = item.find("span", {'class': 'infoBed'}).find('b').text
        except:
            d['Beds'] = None

        try: 
            d['Area'] = item.find('span', {'class' : 'infoSqFt'}).find('b').text
        except:
            d['Area'] = None

        try: 
            d['Full Bath'] = item.find('span', {'class' : 'infoValueFullBath'}).find('b').text
        except:
            d['Full Bath'] = None

        try: 
            d['Half Bath'] = item.find('span', {'class' : 'infoValueHalfBath'}).find('b').text
        except:
            d['Half Bath'] = None 

        for column_group in item.find_all('div', {'class' : 'columnGroup'}):
            for feature_group, feature_name in zip(column_group.find_all('span', {'class':'featureGroup'}), column_group.find_all('span', {'class':'featureName'})):
                if 'Lot Size' in feature_group.text:
                    d['Lot Size'] = feature_name.text
        list.append(d)
    #print(list)

http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=0.html
http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=10.html
http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=20.html


In [4]:
# Creating dataframe to store info 

df = pd.DataFrame(list)
print(df)

         Price                        Address                     Locality  \
0     $725,000                      0 Gateway       Rock Springs, WY 82901   
1     $452,900          1003 Winchester Blvd.       Rock Springs, WY 82901   
2     $396,900                  600 Talladega       Rock Springs, WY 82901   
3     $389,900             3239 Spearhead Way       Rock Springs, WY 82901   
4     $254,000             522 Emerald Street       Rock Springs, WY 82901   
5     $252,900           1302 Veteran's Drive       Rock Springs, WY 82901   
6     $210,000               1021 Cypress Cir       Rock Springs, WY 82901   
7     $209,000                 913 Madison Dr       Rock Springs, WY 82901   
8     $199,900              1344 Teton Street       Rock Springs, WY 82901   
9     $196,900                 4 Minnies Lane       Rock Springs, WY 82901   
10  $1,700,000                  9339 Sd 26900        Rocksprings, TX 78880   
11  $1,100,000               RR674P13 Hwy 377        Rocksprings

In [5]:
# Saving to csv 
#df.to_csv(df)